In [ ]:
import numpy as np
import torch
from torch.optim import lr_scheduler
import gym
from agent import PPO
import os, shutil
from datetime import datetime
import matplotlib.pyplot as plt

### Helper functions

In [ ]:
def evaluate_policy(env, model, render, steps_per_epoch):
    scores = 0
    turns = 3
    for j in range(turns):
        s, done, ep_r, steps = env.reset(), False, 0, 0
        while not (done or (steps >= steps_per_epoch)):
            # Take deterministic actions at test time
            a, logprob_a = model.evaluate(s)
            act = lambda a: 2 * (a-0.5)*env.action_space.high[0]
            s_prime, r, done, info = env.step(act(a))

            ep_r += r
            steps += 1
            s = s_prime
            if render:
                env.render()
        scores += ep_r
    return scores/turns


def plot_learning_curve(x, scores):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    
    plt.plot(x, running_avg, label="Running average")
    plt.plot(x, scores, alpha=0.4)
    plt.title('Learning plot')
    plt.xlabel("Runs")
    plt.ylabel("Scores")
    plt.legend(loc="best")

### Main training

In [ ]:
def main():

    # Environment
    random_seed = 0
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    env = gym.make("LunarLanderContinuous-v2")
    env.seed(random_seed)
    
    # Evaluation environment
    eval_env = gym.make("LunarLanderContinuous-v2")
    eval_env.seed(random_seed)
    

    # Hyperparameters
    kwargs = {
        "input_dims": env.observation_space.shape, 
        "n_actions": env.action_space.shape[0], 
        "gamma": 0.99, 
        "gae_lambda": 0.95, 
        "policy_clip": 0.2, 
        "n_epochs": 10, 
        "net_width": 256, 
        "a_lr": 3e-4, 
        "c_lr": 3e-4, 
        "l2_reg": 1e-3, 
        "dist": 'GS',
        "batch_size": 64, 
        "entropy_coef": 1e-3, 
        "entropy_coef_decay": 0.99
    }

    N = 2048 # lenth of long trajectory
    max_steps = env._max_episode_steps
    Max_train_steps = 500e3
    save_interval = 100e3
    eval_interval = 10e3
    best_interval = 350e3


    if not os.path.exists('model'): 
        os.mkdir('model')
    
    if not os.path.exists('best_model'): 
        os.mkdir('best_model')
    
    model = PPO(**kwargs)
   
 
    traj_lenth = 0
    total_steps = 0
    score_history = []
    
    while total_steps < Max_train_steps:
        observation, done, steps, ep_r = env.reset(), False, 0, 0

        '''Interact & trian'''
        while not done:
            traj_lenth += 1
            steps += 1

            action, prob = model.select_action(observation)
            
            act = lambda a: 2 * (a-0.5)*env.action_space.high[0]  #Action adapter [0,1] to [-max,max]
            observation_, reward, done, info = env.step(act(action))
            model.remember(observation, observation_, action, prob, reward, done)

            observation = observation_
            ep_r += reward

           
            if traj_lenth % N == 0:
                model.train()
                traj_lenth = 0
            
            '''record & log'''
            if total_steps % eval_interval == 0:
                score = evaluate_policy(eval_env, model, False, max_steps)
                score_history.append(score)
                print('EnvName: Lunar-Lander','steps: {}k'.format(int(total_steps/1000)),'score:', score)

            
            total_steps += 1

            '''save model'''
            if total_steps % save_interval==0:
                model.save(total_steps)
            

            ''' best model '''
            if total_steps >= best_interval:
                if score_history[-1] > score_history[-2]:
                    model.best_save()


    env.close() 
    
    x = [i+1 for i in range(len(score_history))]
    plot_learning_curve(x, score_history)

if __name__ == '__main__':
    main()

### Model testing

In [ ]:
# Hyperparameters
env = gym.make("LunarLanderContinuous-v2")

kwargs = {
    "input_dims": env.observation_space.shape, 
    "n_actions": env.action_space.shape[0], 
    "gamma": 0.99, 
    "gae_lambda": 0.95, 
    "policy_clip": 0.2, 
    "n_epochs": 10, 
    "net_width": 256, 
    "a_lr": 3e-4, 
    "c_lr": 3e-4, 
    "l2_reg": 1e-3, 
    "dist": 'GS',
    "batch_size": 64, 
    "entropy_coef": 0, 
    "entropy_coef_decay": 0.9998
}


model = PPO(**kwargs)

model.load_best()
scores = []


for i in range(20):
    obs = env.reset()
    actions = []
    score = 0
    while True:
        action, prob = model.evaluate(obs)
        act = lambda a: 2 * (a-0.5)*env.action_space.high[0]  #Action adapter [0,1] to [-max,max]
        obs, reward, done, info = env.step(act(action))
        score += reward
        actions.append(action)

        if done:
            print(f"Done, points: {score}")
            break
    
    scores.append(score)

print(f"Mean score: {np.mean(scores)}")